In [ ]:
import json
import re
import collections
import os
from pathlib import Path

def extract_hanja(text):
    """
    텍스트에서 한자(CJK Unified Ideographs)만 추출
    """
    # CJK Unified Ideographs 범위: U+4E00-U+9FFF
    hanja_pattern = re.compile(r'[\u4e00-\u9fff]')
    hanja_chars = hanja_pattern.findall(text)
    return ''.join(hanja_chars)

def generate_ngrams(text, n):
    """
    문자열로부터 n-그램 생성
    """
    return [text[i:i+n] for i in range(len(text) - n + 1)]

def process_text_file(file_path):
    """
    텍스트 파일 처리
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        try:
            with open(file_path, 'r', encoding='cp949') as f:
                content = f.read()
        except Exception as e:
            print(f"파일 읽기 오류 ({file_path}): {e}")
            return collections.Counter()

    # 한자만 추출
    all_hanja = extract_hanja(content)

    if not all_hanja:
        print(f"'{file_path}'에서 한자를 찾을 수 없습니다.")
        return collections.Counter()

    print(f"'{file_path}' - 추출된 한자 수: {len(all_hanja)}")

    # 2-그램 생성
    bigrams = generate_ngrams(all_hanja, 2)

    if not bigrams:
        print(f"'{file_path}' - 생성된 2-그램이 없습니다.")
        return collections.Counter()

    # 빈도수 계산
    bigram_counts = collections.Counter(bigrams)

    return bigram_counts

def save_bigrams_to_file(bigram_counts, file_path='hanja_bigram_counts_all.txt'):
    """모든 2-그램 빈도수를 파일로 저장"""
    if not bigram_counts:
        print("저장할 데이터가 없습니다.")
        return

    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(f"총 고유 바이그램 수: {len(bigram_counts)}\n")
            f.write(f"총 바이그램 빈도 합계: {sum(bigram_counts.values())}\n")
            f.write("-" * 50 + "\n")

            for bigram, count in bigram_counts.most_common():
                f.write(f"{bigram}: {count}\n")
        print(f"모든 결과가 '{file_path}'에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 오류: {e}")

def main():
    # 텍스트 파일들이 있는 디렉토리 경로 (현재 디렉토리로 설정)
    directory_path = '.'  # 현재 디렉토리, 필요시 경로 변경

    # 전체 바이그램 카운터
    total_bigram_counts = collections.Counter()

    # 처리된 파일 수 카운트
    processed_files = 0

    # 디렉토리의 모든 .txt 파일 처리
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            print(f"\n'{filename}' 파일 처리 중...")

            # 각 파일의 바이그램 처리
            file_bigram_counts = process_text_file(file_path)

            if file_bigram_counts:
                total_bigram_counts.update(file_bigram_counts)
                processed_files += 1
                print(f"'{filename}' - 고유 바이그램 수: {len(file_bigram_counts)}")

    if processed_files == 0:
        print("처리할 .txt 파일을 찾을 수 없습니다.")
        return

    if not total_bigram_counts:
        print("전체적으로 처리할 데이터가 없습니다.")
        return

    print(f"\n=== 전체 결과 요약 ===")
    print(f"처리된 파일 수: {processed_files}")
    print(f"총 고유 바이그램 수: {len(total_bigram_counts)}")
    print(f"총 바이그램 빈도 합계: {sum(total_bigram_counts.values())}")

    # 상위 20개 바이그램 콘솔 출력 (전체는 파일로 저장)
    print(f"\n가장 흔한 한자 2-그램 (상위 20개):")
    for i, (bigram, count) in enumerate(total_bigram_counts.most_common(20), 1):
        print(f"{i:2d}. {bigram}: {count}")

    # 모든 결과를 파일로 저장
    save_bigrams_to_file(total_bigram_counts)

    print(f"\n전체 {len(total_bigram_counts)}개의 바이그램이 파일에 저장되었습니다.")

if __name__ == "__main__":
    main()
